In [1]:
import pandas as pd

In [2]:
import tensorflow as tf

# Check TensorFlow version
print(f"TensorFlow version: {tf.__version__}")


TensorFlow version: 2.10.0


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [4]:
# Check PyTorch version
print(f"PyTorch version: {torch.__version__}")


PyTorch version: 2.3.1+cpu


## Loading the Data

In [5]:
corpus = ["apple banana fruit", "banana apple fruit", "banana fruit apple",
          "dog cat animal", "cat animal dog", "cat dog animal"]

In [6]:
#1. tokenization by each word
corpus = [sent.split(" ") for sent in corpus]
corpus

[['apple', 'banana', 'fruit'],
 ['banana', 'apple', 'fruit'],
 ['banana', 'fruit', 'apple'],
 ['dog', 'cat', 'animal'],
 ['cat', 'animal', 'dog'],
 ['cat', 'dog', 'animal']]

In [7]:
#2. numeralization
flatten = lambda l: [item for sublist in l for item in sublist] #function to concat lists
vocabs = list(set(flatten(corpus))) #find and store unique words

In [8]:
vocabs

['fruit', 'animal', 'banana', 'dog', 'apple', 'cat']

In [9]:
word2index = {val:index for index, val in enumerate(vocabs)} #create dict with words and their indexes

In [10]:
word2index

{'fruit': 0, 'animal': 1, 'banana': 2, 'dog': 3, 'apple': 4, 'cat': 5}

In [11]:
#The "<UNK>" token stands for "unknown" and is used to handle words that are not in our vocabulary
vocabs.append('<UNK>')
word2index['<UNK>'] = 6

In [12]:
#creating reverse dictionary to map indices back to words
index2word = {word:index for index, word in word2index.items()}
index2word[5]

'cat'